Задание 1.
( 6 баллов) Напишите функцию, которая моделирует один нейрон с сигмоидной активацией и реализует вычисление градиента для обновления весов и смещений нейрона. Функция должна принимать список векторов признаков, ассоциированные бинарные метки класса, начальные веса, начальное смещение, скорость обучения и количество эпох. Функция должна обновлять веса и смещение с помощью градиентного спуска (классической версии) на основе функции потерь NLL и возвращать обновленные веса, смещение и список значений NLL для каждой эпохи, округленное до четырех десятичных знаков. Проведите обучение на предоставленном наборе данных из задания 4 (для двух разных лет). Опционально сгенерируйте другие подходящие наборы данных. Опишите ваши результаты. Предоставленная функция будет также протестирована во время защиты ДЗ. Можно использовать только чистый torch (без использования autograd и torch.nn)

In [127]:
import torch

def sigmoid(x, epsilon=0):
    return torch.clamp((1 / (1 + torch.exp(-x))), min=epsilon, max=1-epsilon)


def nll_loss(y_pred, y_true):
    return -(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred)).mean()

def gradient_descent(weights, bias, dw, db, learning_rate=0.1):
    with torch.no_grad():
        weights -= learning_rate * dw
        bias -= learning_rate * db
    return weights, bias
    
def train_neuron(X, y, initial_weights, initial_bias, learning_rate=0.01, epochs=10, loss_fn=nll_loss, optimizer_fn=gradient_descent, epsilon=0):
    loss_values = []
    weights = initial_weights.clone().detach().requires_grad_(True)
    bias = initial_bias.clone().detach().requires_grad_(True)
    m_weights = torch.zeros_like(weights)
    m_bias = torch.zeros_like(bias)
    v_weights = torch.zeros_like(weights)
    v_bias = torch.zeros_like(bias)
    for epoch in range(epochs):
        # Forward pass
        z = torch.matmul(X, weights) + bias
        y_pred = sigmoid(z, epsilon)
        loss = loss_fn(y_pred, y)
        loss_values.append(round(loss.item(), 4))
        
        # Backward pass
        loss.backward()
        # weights, bias = gradient_descent(weights, bias, torch.matmul(X.t(), (y_pred - y)) / len(X), (y_pred - y).mean(), learning_rate)
        weights, bias, m_weights, m_bias, v_weights, v_bias = optimizer_fn(weights, bias, m_weights, m_bias, v_weights, v_bias, epoch+1, learning_rate)

        # Zero the gradients
        weights.grad.zero_()
        bias.grad.zero_()

    return weights, bias, loss_values

In [128]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

X = pd.read_csv('train_x.csv')
y = pd.read_csv('train_y.csv')
X = X.drop(X.columns[0], axis=1)
y = y.drop(y.columns[0], axis=1)

years_to_use = [2006, 2007]
mask = y['year'].isin(years_to_use)
X = X[mask]
y = y[mask]

year_mapping = {2006: 0, 2007: 1}
y['year'] = y['year'].map(year_mapping)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).squeeze()

initial_weights = torch.zeros(X.shape[1], requires_grad=True)
initial_bias = torch.tensor(0.0, requires_grad=True)
learning_rate = 0.5
epochs = 10
eps = 1e-5

In [93]:
updated_weights, updated_bias, nll_values = train_neuron(X, y, initial_weights, initial_bias, learning_rate, epochs, epsilon=eps)

print("Updated weights:", updated_weights)
print("Updated bias:", updated_bias)
print("NLL values:", nll_values)

Updated weights: tensor([-2.7537e-02, -3.0597e-03,  1.8042e-02,  6.1462e-03, -6.4874e-02,
         2.0049e-02,  2.6663e-02, -3.1438e-03,  3.4858e-03,  7.1858e-04,
        -4.6254e-02, -2.0814e-02,  1.1613e-02,  3.0140e-02,  3.3262e-03,
         1.1781e-02,  3.2995e-03,  8.3393e-03,  2.8283e-02, -3.4848e-02,
         1.1471e-02,  4.5568e-04,  3.2473e-03, -1.0949e-02, -1.8882e-02,
         1.9587e-02, -2.2203e-02,  1.7279e-02, -4.9612e-03, -3.6055e-02,
         2.4571e-02, -1.2637e-02,  1.4778e-02, -5.0098e-04, -4.8418e-03,
         1.6184e-02,  2.8679e-02,  2.4067e-02,  3.2423e-02,  4.6086e-02,
        -2.4088e-02,  2.7174e-02, -1.2515e-02,  3.9274e-02, -1.4684e-02,
        -3.0494e-02,  2.5081e-03, -1.4940e-02,  2.8401e-02, -7.4540e-03,
         2.3903e-02,  6.1825e-04,  2.5888e-02,  2.2094e-02,  1.7626e-02,
         7.9589e-03,  1.9820e-02, -5.8408e-03, -4.9160e-02,  4.2695e-02,
        -2.0137e-02, -1.7455e-03, -3.8323e-02,  2.3120e-02,  1.5536e-02,
        -1.3640e-02, -2.7970e-02, 

Задание 2. {*}
(10 баллов) Реализуйте базовые функции autograd. Можете вдохновиться видео от Andrej Karpathy. Напишите класс, аналогичный предоставленному классу 'Element', который реализует основные операции autograd: сложение, умножение и активацию ReLU. Класс должен обрабатывать скалярные объекты и правильно вычислять градиенты для этих операций посредством автоматического дифференцирования. Плюсом будет набор предоставленных тестов, оценивающих правильность вычислений. Большим плюсом будет, если тесты будут написаны с помощью unittest. Можно использовать только чистый torch (без использования autograd и torch.nn). За каждую нереализованную операцию будет вычитаться 3 балла. Пример: a = Element(2) b = Element(-3) c = Element(10) d = a + b * c e = d.relu() e.backward() print(a, b, c, d, e) Output: Element(data=2, grad=0) Element(data=-3, grad=10) Element(data=10, grad=-3) Element(data=-28, grad=1) Element(data=0, grad=1)

In [1]:
class Element:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0  # Градиент элемента (изначально равен 0)
        self._backward = lambda: None  # Функция для вычисления градиента
        self._prev = set(_children)  # Множество предыдущих элементов
        self._op = _op  # Операция, которая была применена для получения текущего элемента

    def __repr__(self):
        # Возвращает строковое представление элемента, включающее его данные и градиент
        return f"Element(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        # Определяет операцию сложения для элементов
        if not isinstance(other, Element):  # Если второе слагаемое другого типа, операция не определена
            return NotImplemented

        def _backward():
            # Функция для вычисления градиента при сложении
            self.grad += 1 * other.grad
            other.grad += 1 * self.grad

        # Создаем новый элемент, который является суммой двух элементов
        result = Element(self.data + other.data, (self, other), _op='+')
        result._backward = _backward
        return result

    def __mul__(self, other):
        # Определяет операцию умножения для элементов
        if not isinstance(other, Element):  # Если второй множитель другого типа, операция не определена
            return NotImplemented

        def _backward():
            # Функция для вычисления градиента при умножении
            self.grad += other.data * self.grad
            other.grad += self.data * self.grad

        # Создаем новый элемент, который является произведением двух элементов
        result = Element(self.data * other.data, (self, other), _op='*')
        result._backward = _backward
        return result

    def relu(self):
        # Применяет функцию ReLU к элементу
        def _backward():
            # Функция для вычисления градиента при применении ReLU
            self.grad += 1 * (self.data > 0) * self.grad

        # Создаем новый элемент, который является результатом применения ReLU
        result = Element(max(0, self.data), (self,), _op='relu')
        result._backward = _backward
        return result

    def backward(self):
        # Метод для вычисления градиентов
        self.grad += 1  # Инициализируем градиент текущего элемента как 1
        visited = set()  # Множество посещенных элементов
        nodes = [self]  # Список элементов для обработки (первый элемент - текущий)

        while nodes:
            node = nodes.pop()  # Извлекаем элемент из списка
            if node not in visited:
                visited.add(node)  # Добавляем элемент в множество посещенных
                node._backward()  # Вычисляем градиент для текущего элемента
                nodes.extend(node._prev)  # Добавляем предыдущие элементы в список для обработки


In [2]:
a = Element(2)
b = Element(-3)
c = Element(10)

d = a + b * c
e = d.relu()
e.backward()

print(a)
print(b)
print(c)
print(d)
print(e)


Element(data=2, grad=0)
Element(data=-3, grad=0)
Element(data=10, grad=0)
Element(data=-28, grad=0)
Element(data=0, grad=1)


Задание 3.
Реализуйте один из оптимизаторов на выбор. Придумайте и напишите тесты для проверки выбранного оптимизатора. Проведите обучение нейрона из первого задания с использованием оптимизатора, а не ванильного градиентного спуска. Также опишите идею алгоритма (+1 балл). {*} Можете реализовать более 1 алгоритма. Каждый следующий даст 1 балл.

https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
https://arxiv.org/abs/1711.05101

In [132]:
def adam(weights, bias, m_weights, m_bias, v_weights, v_bias, t, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.01):
    weights_grad = weights.grad
    bias_grad = bias.grad
        
    m_weights = beta1 * m_weights + (1 - beta1) * weights_grad
    m_bias = beta1 * m_bias + (1 - beta1) * bias_grad
    v_weights = beta2 * v_weights + (1 - beta2) * weights_grad**2
    v_bias = beta2 * v_bias + (1 - beta2) * bias_grad**2
    
    m_hat_weights = m_weights / (1 - beta1**t)
    m_hat_bias = m_bias / (1 - beta1**t)
    v_hat_weights = v_weights / (1 - beta2**t)
    v_hat_bias = v_bias / (1 - beta2**t)

    with torch.no_grad():
        # weight decay
        weights *= (1 - lr * weight_decay)
        bias *= (1 - lr * weight_decay)
        
        weights -= lr * m_hat_weights / (torch.sqrt(v_hat_weights) + eps)
        bias -= lr * m_hat_bias / (torch.sqrt(v_hat_bias) + eps)
    return weights, bias, m_weights, m_bias, v_weights, v_bias

initial_weights = torch.zeros(X.shape[1], requires_grad=True)
initial_bias = torch.tensor(0.0, requires_grad=True)
updated_weights, updated_bias, nll_values = train_neuron(X, y, initial_weights, initial_bias, learning_rate=0.05, epochs=50, optimizer_fn=adam, epsilon=eps)
print("Updated weights:", updated_weights)
print("Updated bias:", updated_bias)
print("NLL values:", nll_values)

Updated weights: tensor([-9.7896e-02,  1.3599e-02,  2.4764e-02,  5.7589e-02, -1.0610e-01,
        -2.2773e-02,  2.2934e-02, -1.0068e-02,  2.4557e-02, -7.3225e-02,
        -1.2689e-01, -3.0651e-02, -5.9331e-03,  9.5473e-02, -5.5735e-02,
         3.5051e-02, -7.6120e-03,  6.7291e-02,  8.5745e-02, -2.3933e-01,
         6.4303e-02,  7.6357e-02,  1.3402e-02, -4.5313e-02, -5.2229e-02,
         3.7564e-02, -1.2236e-01,  4.5275e-02, -1.9747e-02, -1.0032e-01,
         8.3689e-03, -9.3994e-03,  2.4281e-03,  2.0051e-04, -2.1780e-02,
         2.3621e-02,  6.8958e-02,  3.1565e-02,  9.1768e-02,  8.5176e-02,
        -6.0717e-02,  1.3983e-01, -6.0379e-02,  8.7173e-02, -4.7965e-02,
        -2.9013e-02, -1.7942e-02,  4.9713e-02,  8.1554e-02, -4.4766e-02,
         8.1845e-02, -2.4049e-02,  5.7293e-02,  1.9475e-02,  3.6777e-02,
         2.4629e-02,  6.5339e-02, -1.2626e-02, -9.5825e-02,  2.0477e-01,
        -1.2376e-02,  1.9611e-02, -1.0352e-01,  4.1156e-02,  1.0382e-01,
        -5.3978e-02, -2.4556e-02, 